# 0. Env

In [ ]:
import os

import numpy as np
import pandas as pd

from mecab import MeCab
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

from rank_bm25 import BM25Okapi

In [ ]:
# 데이터를 저장할 경로
data_home = "data"

# 1. 형태소 분석기

## 1.1 Mecab

In [ ]:
mecab = MeCab()

In [ ]:
mecab.morphs('영등포구청역에 있는 맛집 좀 알려주세요.')

In [ ]:
mecab.nouns('우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!')

In [ ]:
mecab.pos('자연주의 쇼핑몰은 어떤 곳인가?')

In [ ]:
mecab.parse('즐거운 하루 보내세요!')

## 1.2. KoNLPy

In [ ]:
# for Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"

In [ ]:
# tagger = Hannanum()
# tagger = Kkma()
# tagger = Komoran()
tagger = Okt()

In [ ]:
tagger.morphs('영등포구청역에 있는 맛집 좀 알려주세요.')

In [ ]:
tagger.nouns('우리나라에는 무릎 치료를 잘하는 정형외과가 없는가!')

In [ ]:
tagger.pos('자연주의 쇼핑몰은 어떤 곳인가?')

# 2. TF-IDF
- 참고: https://wikidocs.net/31698

In [ ]:
# TF-IDF 이해를 위한 짧은 문서
corpus = [
    '먹고 싶은 사과',
    '먹고 싶은 바나나',
    '길고 노란 바나나 바나나',
    '저는 과일이 좋아요'
]

In [ ]:
# 전체 문서 수
N = len(corpus)

In [ ]:
# make vocab
words = list(set(w for doc in corpus for w in doc.split()))
words.sort()
vocab = {w:i for i, w in enumerate(words)}
vocab

In [ ]:
def tf(w, d):
  return d.count(w)


def idf(w, corpus):
  df = 0
  for doc in corpus:
    df += w in doc
  return np.log(N/df)


def tfidf(w, d, corpus):
  return tf(w, d)* idf(w, corpus)

In [ ]:
result = []

# 각 문서에 대해서 아래 연산을 반복
for i in range(N):
  result.append([])
  d = corpus[i]
  for w in vocab.keys():
    result[-1].append(tf(w, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

In [ ]:
result = []
for w in vocab.keys():
    result.append(idf(w, corpus))

idf_ = pd.DataFrame(result, index=vocab, columns=["IDF"]).T
idf_

In [ ]:
result = []
for i in range(N):
  result.append([])
  d = corpus[i]
  for w in vocab.keys():
    result[-1].append(tfidf(w, d, corpus))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

# 3. BM25

In [ ]:
# corpus
corpus = [
    "지미 카터\n제임스 얼 “지미” 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령 (1977-81)이다.\n생애.\n어린 시절.\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n정계 입문.\n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n대통령 재임.\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책을 내세워서 많은 지지를 받았으며 제럴드 포드 대통령을 누르고 당선되었다.\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.",
    "수학\n수학(, , math)은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다. 수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파악한다.\n수학의 개념들은 기원전 600년 경에 활동하며 최초의 수학자로도 여겨지는 탈레스의 기록은 물론, 다른 고대 문명들에서도 찾아볼 수 있으며 인류의 문명과 함께 발전해 왔다. 오늘날 수학은 자연과학, 사회과학, 공학, 의학 등 거의 모든 학문에서도 핵심적인 역할을 하며 다양한 방식으로 응용된다.\n수학을 의미하는 mathematics라는 단어는 '아는 모든 것', '배우는 모든 것'이라는 뜻의 고대 그리스어 'máthēma'(μάθημα) 및 그 활용형 mathēmatikós(μαθηματικός)에서 유래되었다.",
    "수학 상수\n수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.\n수학 상수는 대개 실수체나 복소수체의 원소이다. 우리가 이야기할 수 있는 상수는 (거의 대부분 계산 가능한) 정의가능한 수이다.\n특정 수학 상수, 예를 들면 골롬-딕맨 상수, 프랑세즈-로빈슨 상수, formula_1, 레비 상수와 같은 상수는 다른 수학상수 또는 함수와 약한 상관관계 또는 강한 상관관계를 갖는다.",
    "문학\n문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다. 간단하게 설명하면, 언어를 통해 인간의 삶을 미적(美的)으로 형상화한 것이라고 볼 수 있다. 문학은 원래 문예(文藝)라고 부르는 것이 옳으며, 문학을 학문의 대상으로서 탐구하는 학문의 명칭 역시 문예학이다. 문예학은 음악사학, 미술사학 등과 함께 예술학의 핵심분야로서 인문학의 하위범주에 포함된다.\n일반적으로 문학의 정의는 텍스트들의 집합이다. 각각의 국가들은 고유한 문학을 가질 수 있으며, 이는 기업이나 철학 조류, 어떤 특정한 역사적 시대도 마찬가지이다. 흔히 한 국가의 문학을 묶어서 분류한다. 예를 들어 고대 그리스어, 성서, 베오울프, 일리아드, 그리고 미국 헌법 등이 그러한 분류의 범주에 들어간다. 좀 더 일반적으로는 문학은 특정한 주제를 가진 이야기, 시, 희곡의 모음이라 할 수 있다. 이 경우, 이야기, 시, 그리고 희곡은 민족주의적인 색채를 띨 수도 아닐 수도 있다. 문학의 한 부분으로서 특정한 아이템을 구분 짓는 일은 매우 어려운 일이다. 어떤 사람들에게 \"문학\"은 어떠한 상징적인 기록의 형태로도 나타날 수 있는 것이다. (이를테면 이미지나 조각, 또는 문자로도 나타날 수 있다.) 그러나 또다른 사람들에게 있어 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 형태를 가진 텍스트여야 하고, 대개 그러한 형태는 종이 등의 눈에 보이는 매체에서 디지털 미디어까지 다양할 수 있다.",
    "화학\n화학(化學)은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학(自然科學)의 한 분야이다. 물리학(物理學)도 역시 물질을 다루는 학문이지만, 물리학이 원소(元素)와 화합물(化合物)을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을 다루고 이러한 현상으로부터 통일된 이론을 구축하려는 것과는 달리 화학에서는 물질 자체를 연구 대상으로 한다. 화학은 이미 존재하는 물질을 이용하여 특정한 목적에 맞는 새로운 물질을 합성하는 길을 제공하며, 이는 농작물(農作物)의 증산, 질병의 치료 및 예방, 에너지 효율 증대, 환경오염(環境汚染) 감소 등 여러 가지 이점을 제공한다.\n어원.\n화학은 연금술사들이 물질을 섞으며 발전시켰기 때문에 화학을 뜻하는 영어 ‘케미스트리(chemistry)’는 연금술을 뜻하는 단어 ‘알케미(alchemy)’에서 비롯하였다. 이는 다시 아랍어 ‘알 키미야(, al-kīmiyāʾ)’에서 왔는데, 이 단어의 어원에 대해서는 여러 가지 설이 있다.\n‘화학(化學)’이란 단어는 물질의 변화를 다루는 학문이라는 점에 착안한 번역어이다. 이 번역어는 의 《항해술기(航海述奇)》(1866), 의 자연과학 교과서 《격물입문(格物入門)》(1866) 등에서 처음 쓰였다.\n역사.\n고대 화학(古代化學)",
]

In [ ]:
# 형태소 분석기를 이용한 tokeinizer 선언
# 조사 등 일부 품사를 제거
# 품사표: https://blog.naver.com/aramjo/221404488280
MECAB = MeCab()
EXCLUDE = set(['JKS', 'JKC', 'JKG', 'JKO', 'JKB',
               'JKV', 'JKQ', 'JX', 'JC', 'EP',
               'EF', 'EC', 'ETN', 'ETM',
               'SF', 'SSC', 'SSO', 'SY'])
def tokenizer(sent):
    tokens = []
    for w, t in MECAB.pos(sent):
        if t not in EXCLUDE:
            tokens.append(w)
    return tokens

In [ ]:
# tokenize
tokenized_chunks = [tokenizer(chunk) for chunk in corpus]
print(tokenized_chunks[0])

In [ ]:
# bm25 class 생성
bm25 = BM25Okapi(tokenized_chunks)

In [ ]:
# 문서의 길이 확인
bm25.doc_len

In [ ]:
# TF
bm25.doc_freqs

In [ ]:
# IDF
bm25.idf

In [ ]:
# query tokenize
query = "수학의 정의가 뭐야?"
tokenized_query = tokenizer(query)
tokenized_query

In [ ]:
# score 계산
scores = bm25.get_scores(tokenized_query)
scores

In [ ]:
# score 순서로 정렬
rank = np.argsort(-scores)
rank

## 연습문제
- bm25 query를위해서 query_bm25 함수를 완성하세요.

In [ ]:
def query_bm25(bm25, query, tokenizer, top_n=10):
    # input tokenize
    # score 계산
    # score 순서로 정렬

    # top-n 출력
    result = []
    for i in rank[:top_n]:
        result.append((i, scores[i]))
    return result

In [ ]:
while True:
    query = input('검색 > ')
    query = query.strip()
    # 입력값이 없으면 종료
    if len(query) == 0:
        break
    # bm25 query
    result = query_bm25(bm25, query, tokenizer)
    # 결과 출력
    for i, score in result:
        print(f'---- score: {score} ----')
        print(corpus[i])
        print()